'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3  
SQLALCHEMY VERSION: 1.1.13

'''

### Pollutant Constituents
Below are the pollutant constituents we attempt to address through this alternatives analysis

In [1]:
#############################################################################################################
#                   
#                                       DEFINE GLOBAL VARIABLE pollLS
#############################################################################################################     
pollLS = ['tss', 'turbidity', 'p', 'n', 'nn', 'an', 'og', 'cu', 'zn', 'fe', 'phmin', 'phmax'] 

# Program Setup
## (Importing libraries, defining database)

In [2]:
#import standard python libraries:
import winsound
import pandas as pd
import numpy as np
import math
import datetime
import calendar
import time
import itertools
import random

In [3]:
#IMPORT AND DEFINE sqlalchemy libraries, tables, and session engine
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_existing_pollutant_concentrations import Existing_Pollutant_Concentrations as ExPollConcs
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    {TableName:Lambda Function, TableName:Lambda Function, ...}
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function is a SQLAlchemy query used to test record uniqueness. The function can take on any form 
        but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
                   CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
    FALSE: indicates that db table doesn't impose uniqueness on its records (other than its record id being unique)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['nel_column'],
    'existing_pollutant_concentrations': False, #uniqueness not imposed for records in this table.
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions


Clearing old DB


In [4]:
'''
Define other custom modules
'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_EffluentLimit as EffLim
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


#  Import Data

In [5]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece2.csv') 

#IMPORT BASIC FACILITY CHARS:
    #!!!!IMPORTANT!!!! This import must occur before other facility specific data is imported!
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS EXISTANCE FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
#IF CSV HEADRS SETUP CORRECTLY, THEN THIS INSERTS NEL EXISTANCE DATA (0 OR 1) TO WRS_POLLUTANT TABLE 
#AND USES THE FACILITY_TYPE_HAS_NEL TO ASSOCIATE RECORD WITH FACILITY TYPE
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
    #The current process inserts fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)

# #IMPORT FACILITY SAMPLING DATA
 #!!!IMPORTANT!!!! For now, we make none detects = 0 BUT this must be changed to detection limit, per DOH guidance.
print ('\nImporting Facilty Sampling data:')
importCSV.importCSV('Input_Files\\sample_data.csv', unqTests)


# for now, since we're developing, delete out all except 1st 2 facilities.
# n = 5
# session.query(ExPollConcs).filter(ExPollConcs.facility_id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.query(Facility_Chars).filter(Facility_Chars.id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.commit #we chose not to sync session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
var|OFFSITE_SD_Exist
True
var|OFFSITE_SD_Exist
WriteToField:  OFFSITE_SD_Exist  USEDECVAL IN:  ['var', 'OFFSITE_SD_Exist']
val|'Yes'
True
val|'Yes'
WriteToField:  'Yes'  USEDECVAL IN:  ['val', "'Yes'"]

Reading csv record: Feas-2
var|GW_Risk
True
var|GW_Risk
WriteToField:  GW_Risk  USEDECVAL IN:  ['var', 'GW_Risk']
val|'High'
True
val|'High'
WriteToField:  'High'  USEDECVAL IN:  ['val', "'High'"]

Reading csv record: Feas-3
var|GW_Risk
True
var|GW_Risk
WriteToField:  GW_Risk  USEDECVAL IN:  ['var', 'GW_Risk']
val|'High'
True
val|'High'
WriteToField:  'High'  USEDECVAL IN:  ['val', "'High'"]

Reading csv record: Feas-4
var|Soil_Type
True
var|Soil_Type
WriteToField:  Soil_Type  USEDECVAL IN:  ['var', 'Soil_Type']
val|'Rock'
True
val|'Rock'
WriteToField:  'Rock'  USEDECVAL IN:  ['val', "'Rock'"]

Reading csv record: Feas-5
var|Soil_Type
True
var|Soil_Type
WriteToField:  Soil_Type  USEDECVAL IN:  ['var', 'Soil_Type

Added feasibility test def as record:  13
Added feasibility test def as record:  14
Added feasibility test def as record:  15
Added feasibility test def as record:  16
Added feasibility test def as record:  17
Added feasibility test def as record:  18

Reading csv record: Inlet Insert Unit
var|Count_CB
True
var|Count_CB
WriteToField:  Count_CB  USEDECVAL IN:  ['var', 'Count_CB']
{'Count_CB': ['Count_CB', 'val', 'facility_chars', 'Count_CB', 'id', 'FLOAT']}
var|Count_CB
True
var|Count_CB
WriteToField:  Count_CB  USEDECVAL IN:  ['var', 'Count_CB']
{'Count_CB': ['Count_CB', 'val', 'facility_chars', 'Count_CB', 'id', 'FLOAT']}
N/A
True
N/A
{}
Reading pollutant removal rate info...
Linking feasibility tests w/ base bmp: 6
Removed:  0  old feasibility test defs for the base bmp
Added feasibility test def as record:  19

Reading csv record: Sand Filtration
var|Indus_WQV
True
var|Indus_WQV
WriteToField:  Indus_WQV  USEDECVAL IN:  ['var', 'Indus_WQV']
{'Indus_WQV': ['Indus_WQV', 'val', 'facilit

# Existing Sampling Data
Talk about it...

Global variables related to existing sampling data include:  
 - 
 - 

Defined several functions that will be used by BMP Option Evaluation. These include:  
 - 
 - 

In [6]:
'''
#############################################################################################################
#              ASSIGN CONCENTRATION DATA FOR FACILITIES WITHOUT SAMPLING RESULTS:
#                      assignment made into database table: ExPollConcs 
#############################################################################################################
Enter estimated pollutant concentrations into database's existing pollutant concentration table for facilities without 
actual sampling data. Use 1 of 2 methods:

Method 1 (sim_MaxType): Use maximum concentration value sampled for period 2013-2017
          This method is for Permit Table 1 facilities only
          Method assumes we have already entered sampling data for into the database's existing pollutant concentration table

Method 2 (sim_EMC): Use data from an EMC study.
          This method is for facilities that are not on Permit Table 1
'''

def WriteSampleDat_simMaxType(pollLS):
    #assign maximum sampled values to Table 1 facilities that have not yet been sampled
    #delete all pollutant concentration table records that are not from infield sampling.
    #     To be sure we're starting fresh, let's remove any records in ExPollConcs that:
    #     1. Were not obtained directly from field samples (i.e. sample_method != 'infield)
    #     2. Were obtained from field samples, but are not Table 1 facilities (i.e. we shouldn't be looking at their  sample results)
    session.query(ExPollConcs).filter(ExPollConcs.sample_method != 'infield').delete(synchronize_session = False)
    #delete all pollutant concentration table records that are not for Table 1 facilities
    #for some reason bulk delete's not working. so let's use a loop to work around it.
    for rec in session.query(ExPollConcs.id).filter(ExPollConcs.facility_id == Facility_Chars.id).filter(Facility_Chars.Permit_Table != 'Table 1'):
        session.query(ExPollConcs).filter(ExPollConcs.id == rec[0]).delete(synchronize_session = False)

    #make a dataframe called pd_Concs to hold existing pollutant concentrations that were sampled in the field (the 'infield' sampling method)'''
    q = session.query(ExPollConcs).filter(ExPollConcs.sample_method == 'infield')
    pd_Concs = pd.read_sql(q.statement,session.bind)         

    #build pd_infieldExtreama by making a dictionary of maximum sample results for each constiuent
    dict_extrema = {'c_' + Constituent: pd_Concs.loc[:,'c_' + Constituent].max() for Constituent in pollLS}
    dict_extrema['c_phmin'] = pd_Concs.loc[:,'c_phmin'].min() #phMin is exception to above. we want min. phMin value
    #use dictionary to build pd_infieldExtrema dataframe
    pd_infieldExtrema = pd.DataFrame([dict_extrema])
    #     display(pd_infieldExtrema)

    #now build query that identifies all Table 1 facilities that are not in ExPollConcs
    subq = session.query(ExPollConcs.facility_id.distinct()).order_by(ExPollConcs.facility_id).all()
    ls_sq = [i[0] for i in subq if i[0] is not None] #list comprehension to produce list of all facility_id in ExPollConcs table
    #get list of Table 1 facilities not in ExPollConcs:
    tpl_q = session.query(Facility_Chars.id).filter(Facility_Chars.Permit_Table == 'Table 1').filter(Facility_Chars.id.notin_(ls_sq)).all()
    ls_FacIDs = [i[0] for i in tpl_q] #write query tuple to list    
    #make a list of Table 1 facs not in ExPollConcs (a list of dicts). also include extrema conc. values.  
    ls_dict_pd = [{**{'facility_id': FacID, 'sample_method': 'sim_MaxType', 'sample_date':'12/31/2016'}, **dict_extrema} for FacID in ls_FacIDs]
    #write list to database:
    ExPollConcs_meta = Base.metadata.tables['existing_pollutant_concentrations']
    ExPollConcs_id_meta = ExPollConcs_meta.c['id']
    for dict_temp in ls_dict_pd:
        SQLA_main.insertRec(ExPollConcs_meta,dict_temp)
    session.commit()
    #for future implementation: write dict -> dataframe -> db(using sqla):
        # pd_temp.to_sql('existing_pollutant_concentrations', engine, if_exists='append', index = False)
        #http://docs.sqlalchemy.org/en/latest/faq/performance.html#i-m-inserting-400-000-rows-with-the-orm-and-it-s-really-slow
        #https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy

WriteSampleDat_simMaxType(pollLS) #call function defined above   

In [7]:
#############################################################################################################
#                                 Write all sampling data from database to pd_ExConcs
#                                       (DEFINE GLOBAL VARIABLE: pd_ExConcs)
#############################################################################################################    
                  
#get all existing sampling data.
q = session.query(ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_date, 
        ExPollConcs.c_tss,
        ExPollConcs.c_turbidity,
        ExPollConcs.c_p,
        ExPollConcs.c_n,
        ExPollConcs.c_nn,
        ExPollConcs.c_an,
        ExPollConcs.c_og,
        ExPollConcs.c_cu,
        ExPollConcs.c_zn,
        ExPollConcs.c_fe,
        ExPollConcs.c_phmin,
        ExPollConcs.c_phmax  
         ).order_by(ExPollConcs.facility_id) #.filter(ExPollConcs.facility_id == FacID)
pd_ExConcs = pd.read_sql(q.statement,session.bind) 
#tidy up the sampling data
from datetime import datetime
pd_ExConcs['sample_date'] = pd.to_datetime(pd_ExConcs['sample_date'], format="%m/%d/%Y")
#assign NaN values to any None element 
pd_ExConcs = pd_ExConcs.applymap(lambda x: float('nan') if x is None else x) 
print ('a few pieces of data:')
pd_ExConcs

a few pieces of data:


Facility_ID sample_date   c_tss  c_turbidity     c_p     c_n   c_nn  \
0              1  2017-04-19   122.0          NaN     NaN   2.430    NaN   
1              1  2017-02-11    59.0        13.00   0.097   0.580    NaN   
2              1  2016-12-04    80.0        71.20   0.300   0.910  0.120   
3              1  2016-06-17    83.0        81.40   0.250   0.940  0.200   
4              1  2015-02-20    33.5        17.50   0.176   1.830  0.190   
5              1  2014-04-13    14.0         8.50   0.244   2.037  0.247   
6              1  2013-03-09   163.0        24.40   0.155   1.199  0.239   
7              2  2017-04-20     NaN        13.00     NaN   0.780    NaN   
8              2  2017-01-21     0.0        31.00   0.100   1.580  0.000   
9              2  2016-05-05     7.0         4.90   0.066   2.672  0.212   
10             2  2014-06-03    18.5         3.30   0.075   1.136  0.116   
11             2  2013-03-09    80.0         7.80   0.167   0.761  0.061   
12             3  2017-04-28     NaN          NaN     NaN     NaN    NaN   
13             3  2016-12-01   172.0       277.00   1.100   4.320  2.330   
14             3  2015-02-14   311.0        16.20   0.846   4.000  1.060   
15             3  2013-03-14    91.3        42.80   0.262   1.395  0.425   
16             4  2013-04-24   247.0         7.30   0.755   1.349  0.139   
17             5  2017-01-24   249.0        46.00   2.300  15.850  0.000   
18             5  2016-12-08   240.0       767.00   0.137  27.250  0.050   
19             5  2016-05-25    89.5        13.20   2.920  14.542  0.142   
20             5  2013-03-09   132.0        53.00   2.530   0.350  0.050   
21             6  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
22             7  2014-06-03     5.0        18.30   0.028   0.703  0.133   
23             8  2013-03-07   322.0      1410.00   1.330   2.532  0.192   
24             9  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
25            10  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
26            11  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
27            12  2013-03-22    62.5        34.40   0.451   2.647  0.377   
28            13  2017-04-28   280.0       136.00   1.800  21.100  1.500   
29            13  2017-02-06    80.0        73.00   1.200   4.900  0.720   
..           ...         ...     ...          ...     ...     ...    ...   
77           104  2014-05-02    30.5        17.40   0.161   1.152  0.092   
78           104  2013-01-27   233.0       240.00   0.558   1.243  0.203   
79           105  2017-02-11   677.0       720.00   1.500   1.650  1.120   
80           105  2017-02-11   910.0       105.00   0.300   1.080  0.520   
81           105  2013-04-14   303.0        16.00   0.874   1.887  0.097   
82           106  2017-02-06     NaN          NaN     NaN     NaN    NaN   
83           106  2017-02-06   680.0      4000.00   1.200   5.700  3.680   
84           106  2016-12-01   272.0       801.00   0.800   0.700  0.120   
85           106  2016-06-17   151.0         8.15   0.096   0.680  0.000   
86           106  2016-05-25   157.0       137.00   1.600   1.640  0.000   
87           106  2015-02-03   119.0         6.30   0.119   1.296  0.076   
88           106  2015-02-03   155.0         8.20   0.379   1.275  0.085   
89           106  2013-03-09    10.5        17.00   0.124   0.856  0.106   
90           107  2017-04-19     NaN        45.00   0.480   1.680  0.570   
91           107  2017-04-19     NaN        47.00   0.480   1.450  0.470   
92           107  2017-01-24     NaN        90.00   0.500   2.190  0.520   
93           107  2017-01-24   124.0        66.00   0.570   1.940  0.540   
94           107  2016-12-04     7.0         7.20  10.300   0.950  0.200   
95           107  2016-05-14    52.0        31.10   0.281   2.489  0.309   
96           107  2015-02-20    16.0        31.70   0.227   1.412  0.342   
97           107  2013-03-24     6.0         7.80   0

In [8]:
#############################################################################################################
#                                 ESTIMATE Numeric Effluent Limits
#                          (DEFINE GLOBAL VARIABLES: pd_FacsNELs_Wet & pd_FacsNELs_Dry)
#############################################################################################################    
'''
Estimate the Numeric Effluent Limits (NELs) for each facility.
Return wet and dry season NELs in 2 separate dataframes:
    pd_FacsNELs_Wet & pd_FacsNELs_Dry
Estimate NELs using the EffLim module's GetNELs function call.
 The GetNELs function call will differentiate between wet and dry season limits
 (if limits are the same between wet & dry season, then the same limit will be placed into the wet and dry
  dataframes.)
 The GetNEls function calculates a pollutant constituent NEL using this formula:
    NEL = fTypeHas_NEL * SampleClass_NEL
    Where:
      fTypeHas_NEL is a [0,1] value from PBP Table 3, based on facility type (stored in SQLA_DB_facility_type_has_nel)
      SampleClass_NEL is pollutant concentration based on facility's sample class, based on PBP Appendix L
'''
pd_FacsNELs_Wet, pd_FacsNELs_Dry = pd.DataFrame(),  pd.DataFrame() #initialize wet and dry season nel dataframes 
for recFac in session.query(Facility_Chars): #do the following for each facility:
    wet,dry = EffLim.GetNELs(recFac,False) #Get Wed & Dry NELs by calculating: NEL = fTypeHas_NEL * SampleClass_NEL
#     if wet is not None:
    pd_FacsNELs_Wet = pd.concat([pd_FacsNELs_Wet, wet]) #write wet NELs to pd_FacsNELs_Wet
#     if dry is not None:
    pd_FacsNELs_Dry = pd.concat([pd_FacsNELs_Dry, dry]) #write dry NELs to pd_FacsNELs_Dry

print('Wet NELs:')
display(pd_FacsNELs_Wet)
print('Dry NELs:')
display(pd_FacsNELs_Dry)

Wet NELs:


nel_tss  nel_turbidity  nel_p  nel_n  nel_nn  nel_an  nel_og  \
Facility_ID                                                                 
1               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
2               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
3                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
4               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
5               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
6               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
7               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
8                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
9                NaN           0.50   0.03   0.18   0.010  0.0050    15.0   
10              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
11              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
12              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
13              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
14              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
15              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
16               NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
17               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
18              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
19               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
20              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
21               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
22               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
23               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
24              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
25              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
26              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
27              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
28              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
29              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
30               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
...              ...            ...    ...    ...     ...     ...     ...   
90               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
91               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
92               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
93               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
94              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
95              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
96              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
97              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
98               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
99               NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
100             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
101              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
102              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
103             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
104             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
105             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
106              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
107              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
108              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
109      

Dry NELs:


nel_tss  nel_turbidity  nel_p  nel_n  nel_nn  nel_an  nel_og  \
Facility_ID                                                                 
1               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
2               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
3                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
4               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
5               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
6               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
7               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
8                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
9                NaN           0.50   0.03   0.18   0.010  0.0050    15.0   
10              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
11              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
12              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
13              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
14              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
15              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
16               NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
17               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
18              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
19               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
20              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
21               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
22               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
23               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
24              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
25              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
26              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
27              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
28              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
29              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
30               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
...              ...            ...    ...    ...     ...     ...     ...   
90               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
91               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
92               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
93               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
94              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
95              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
96              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
97              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
98               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
99               NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
100             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
101              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
102              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
103             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
104             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
105             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
106              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
107              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
108              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
109      

In [9]:
#############################################################################################################
#                        Define Maximum Concentrations for facility on each sample date
#                                    DEFINE GLOBAL VARIABLE: pd_exMaxConcs
############################################################################################################# 
#(using pd_ExConcs, calculate the maximum concentrations observed at a facility on a given sampling date)
pd_exMaxConcs = pd_ExConcs.groupby('Facility_ID').apply(lambda x: x.groupby('sample_date').agg(np.max).sort_index(ascending=False))
pd_exMaxConcs.drop('Facility_ID', axis = 1, inplace = True) #remove duplicate FAcility_ID column
pd_exMaxConcs.reset_index(1, inplace = True) #remove date index
pd_exMaxConcs.reset_index( inplace = True) #remove facility index
display(pd_exMaxConcs)

Facility_ID sample_date   c_tss  c_turbidity     c_p     c_n   c_nn  \
0              1  2017-04-19   122.0          NaN     NaN   2.430    NaN   
1              1  2017-02-11    59.0        13.00   0.097   0.580    NaN   
2              1  2016-12-04    80.0        71.20   0.300   0.910  0.120   
3              1  2016-06-17    83.0        81.40   0.250   0.940  0.200   
4              1  2015-02-20    33.5        17.50   0.176   1.830  0.190   
5              1  2014-04-13    14.0         8.50   0.244   2.037  0.247   
6              1  2013-03-09   163.0        24.40   0.155   1.199  0.239   
7              2  2017-04-20     NaN        13.00     NaN   0.780    NaN   
8              2  2017-01-21     0.0        31.00   0.100   1.580  0.000   
9              2  2016-05-05     7.0         4.90   0.066   2.672  0.212   
10             2  2014-06-03    18.5         3.30   0.075   1.136  0.116   
11             2  2013-03-09    80.0         7.80   0.167   0.761  0.061   
12             3  2017-04-28     NaN          NaN     NaN     NaN    NaN   
13             3  2016-12-01   172.0       277.00   1.100   4.320  2.330   
14             3  2015-02-14   311.0        16.20   0.846   4.000  1.060   
15             3  2013-03-14    91.3        42.80   0.262   1.395  0.425   
16             4  2013-04-24   247.0         7.30   0.755   1.349  0.139   
17             5  2017-01-24   249.0        46.00   2.300  15.850  0.000   
18             5  2016-12-08   240.0       767.00   0.137  27.250  0.050   
19             5  2016-05-25    89.5        13.20   2.920  14.542  0.142   
20             5  2013-03-09   132.0        53.00   2.530   0.350  0.050   
21             6  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
22             7  2014-06-03     5.0        18.30   0.028   0.703  0.133   
23             8  2013-03-07   322.0      1410.00   1.330   2.532  0.192   
24             9  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
25            10  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
26            11  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
27            12  2013-03-22    62.5        34.40   0.451   2.647  0.377   
28            13  2017-04-28   280.0       136.00   1.800  21.100  1.500   
29            13  2017-02-06    80.0        73.00   1.200   4.900  0.720   
..           ...         ...     ...          ...     ...     ...    ...   
72           103  2017-02-11     NaN        29.00   0.200   0.590    NaN   
73           103  2016-12-08    28.0        33.50   4.800   0.970  0.110   
74           103  2016-06-17    70.0        51.80   0.300   1.050  0.000   
75           103  2014-05-02     8.0        63.60   0.248   1.221  0.161   
76           103  2013-01-27   214.0       189.00   0.468   1.142  0.122   
77           104  2014-05-02    30.5        17.40   0.161   1.152  0.092   
78           104  2013-01-27   233.0       240.00   0.558   1.243  0.203   
79           105  2017-02-11   910.0       720.00   1.500   1.650  1.120   
80           105  2013-04-14   303.0        16.00   0.874   1.887  0.097   
81           106  2017-02-06   680.0      4000.00   1.200   5.700  3.680   
82           106  2016-12-01   272.0       801.00   0.800   0.700  0.120   
83           106  2016-06-17   151.0         8.15   0.096   0.680  0.000   
84           106  2016-05-25   157.0       137.00   1.600   1.640  0.000   
85           106  2015-02-03   155.0         8.20   0.379   1.296  0.085   
86           106  2013-03-09    10.5        17.00   0.124   0.856  0.106   
87           107  2017-04-19     NaN        47.00   0.480   1.680  0.570   
88           107  2017-01-24   124.0        90.00   0.570   2.190  0.540   
89           107  2016-12-04     7.0         7.20  10.300   0.950  0.200   
90           107  2016-05-14    52.0        31.10   0.281   2.489  0.309   
91           107  2015-02-20    16.0        31.70   0.227   1.412  0.342   
92           107  2013-03-24     6.0         7.80   0

In [10]:
#############################################################################################################
#                 Rank each facility's sampled constituent based on sampling date
#                      (earlier sample dates are given lower rank)
############################################################################################################# 
def _HELPER_SampleRank(datetime):
    #return a numeric value for the passed in date format: 2017-02-06 00:00:00
    return int(str(datetime)[:10].replace('-',''))
def _MakeSampleRank(pd_Concs, pollLS):
    #assign sample rank based on date to each constituent type in pollLS.
    #write column of sample dates expressed as numeric value (used later by AF factor. do now b/c only need to setup 1 time)
    #group by facility, then by sample date, then for each facility-sample data pair, use max constituent concentration, 
    #then sort each facility by sample date w/ newest sample first.   
    for Constituent in pollLS:
       #make helper column that expreses date as numeric:
        pd_Concs['c_' + Constituent + '_HelpSR'] = pd_Concs.apply(
            lambda row: _HELPER_SampleRank(row['sample_date']) if not (math.isnan(row['c_'+Constituent])) else np.nan, axis = 1)
        #rank sample dates for each constituent of each facility
        pd_Concs['c_' + Constituent + '_SR'] = pd_Concs.groupby(
            ['Facility_ID'])['c_' + Constituent + '_HelpSR'].rank(ascending = False)-1 #subtract 1 to start ranking at 0. 
        #drop helper column
        pd_Concs = pd_Concs.drop('c_' + Constituent + '_HelpSR', axis = 1)
    return pd_Concs
#############################################################################################################
#                 Rank each facility's sampled constituent based on sampling date
#                      REDEFINE GLOBAL VARIABLE: pd_exMaxConcs
############################################################################################################# 
pd_exMaxConcs = _MakeSampleRank(pd_exMaxConcs, pollLS)
display(pd_exMaxConcs)

Facility_ID sample_date   c_tss  c_turbidity     c_p     c_n   c_nn  \
0              1  2017-04-19   122.0          NaN     NaN   2.430    NaN   
1              1  2017-02-11    59.0        13.00   0.097   0.580    NaN   
2              1  2016-12-04    80.0        71.20   0.300   0.910  0.120   
3              1  2016-06-17    83.0        81.40   0.250   0.940  0.200   
4              1  2015-02-20    33.5        17.50   0.176   1.830  0.190   
5              1  2014-04-13    14.0         8.50   0.244   2.037  0.247   
6              1  2013-03-09   163.0        24.40   0.155   1.199  0.239   
7              2  2017-04-20     NaN        13.00     NaN   0.780    NaN   
8              2  2017-01-21     0.0        31.00   0.100   1.580  0.000   
9              2  2016-05-05     7.0         4.90   0.066   2.672  0.212   
10             2  2014-06-03    18.5         3.30   0.075   1.136  0.116   
11             2  2013-03-09    80.0         7.80   0.167   0.761  0.061   
12             3  2017-04-28     NaN          NaN     NaN     NaN    NaN   
13             3  2016-12-01   172.0       277.00   1.100   4.320  2.330   
14             3  2015-02-14   311.0        16.20   0.846   4.000  1.060   
15             3  2013-03-14    91.3        42.80   0.262   1.395  0.425   
16             4  2013-04-24   247.0         7.30   0.755   1.349  0.139   
17             5  2017-01-24   249.0        46.00   2.300  15.850  0.000   
18             5  2016-12-08   240.0       767.00   0.137  27.250  0.050   
19             5  2016-05-25    89.5        13.20   2.920  14.542  0.142   
20             5  2013-03-09   132.0        53.00   2.530   0.350  0.050   
21             6  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
22             7  2014-06-03     5.0        18.30   0.028   0.703  0.133   
23             8  2013-03-07   322.0      1410.00   1.330   2.532  0.192   
24             9  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
25            10  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
26            11  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
27            12  2013-03-22    62.5        34.40   0.451   2.647  0.377   
28            13  2017-04-28   280.0       136.00   1.800  21.100  1.500   
29            13  2017-02-06    80.0        73.00   1.200   4.900  0.720   
..           ...         ...     ...          ...     ...     ...    ...   
72           103  2017-02-11     NaN        29.00   0.200   0.590    NaN   
73           103  2016-12-08    28.0        33.50   4.800   0.970  0.110   
74           103  2016-06-17    70.0        51.80   0.300   1.050  0.000   
75           103  2014-05-02     8.0        63.60   0.248   1.221  0.161   
76           103  2013-01-27   214.0       189.00   0.468   1.142  0.122   
77           104  2014-05-02    30.5        17.40   0.161   1.152  0.092   
78           104  2013-01-27   233.0       240.00   0.558   1.243  0.203   
79           105  2017-02-11   910.0       720.00   1.500   1.650  1.120   
80           105  2013-04-14   303.0        16.00   0.874   1.887  0.097   
81           106  2017-02-06   680.0      4000.00   1.200   5.700  3.680   
82           106  2016-12-01   272.0       801.00   0.800   0.700  0.120   
83           106  2016-06-17   151.0         8.15   0.096   0.680  0.000   
84           106  2016-05-25   157.0       137.00   1.600   1.640  0.000   
85           106  2015-02-03   155.0         8.20   0.379   1.296  0.085   
86           106  2013-03-09    10.5        17.00   0.124   0.856  0.106   
87           107  2017-04-19     NaN        47.00   0.480   1.680  0.570   
88           107  2017-01-24   124.0        90.00   0.570   2.190  0.540   
89           107  2016-12-04     7.0         7.20  10.300   0.950  0.200   
90           107  2016-05-14    52.0        31.10   0.281   2.489  0.309   
91           107  2015-02-20    16.0        31.70   0.227   1.412  0.342   
92           107  2013-03-24     6.0         7.80   0

In [11]:
#############################################################################################################
#                        Assign NEL for facility sample based on sample date
#                                    REDEFINE  GLOBAL VARIABLE: pd_exMaxConcs
############################################################################################################# 
def _HELPER_GetWetOrDryVal(Constituent, row):
    x = EffLim.Get_pd_NEL_WetOrDry(row['sample_date'], pd_FacsNELs_Wet, pd_FacsNELs_Dry)
    try:
        y = x.loc[row['Facility_ID'], 'nel_'+Constituent]
    except KeyError:
        y = np.nan
    return y
#write nels for each sample based on wet or dry season
for Constituent in pollLS:
    pd_exMaxConcs['nel_'+Constituent] = 0
#     display(pd_exMaxConcs['nel_'+Constituent])
    pd_exMaxConcs['nel_'+Constituent] = pd_exMaxConcs.apply(lambda row: 
      _HELPER_GetWetOrDryVal(Constituent,row), axis = 1)
#     display(pd_exMaxConcs['nel_'+Constituent])
display(pd_exMaxConcs.loc[:])


Facility_ID sample_date   c_tss  c_turbidity     c_p     c_n   c_nn  \
0              1  2017-04-19   122.0          NaN     NaN   2.430    NaN   
1              1  2017-02-11    59.0        13.00   0.097   0.580    NaN   
2              1  2016-12-04    80.0        71.20   0.300   0.910  0.120   
3              1  2016-06-17    83.0        81.40   0.250   0.940  0.200   
4              1  2015-02-20    33.5        17.50   0.176   1.830  0.190   
5              1  2014-04-13    14.0         8.50   0.244   2.037  0.247   
6              1  2013-03-09   163.0        24.40   0.155   1.199  0.239   
7              2  2017-04-20     NaN        13.00     NaN   0.780    NaN   
8              2  2017-01-21     0.0        31.00   0.100   1.580  0.000   
9              2  2016-05-05     7.0         4.90   0.066   2.672  0.212   
10             2  2014-06-03    18.5         3.30   0.075   1.136  0.116   
11             2  2013-03-09    80.0         7.80   0.167   0.761  0.061   
12             3  2017-04-28     NaN          NaN     NaN     NaN    NaN   
13             3  2016-12-01   172.0       277.00   1.100   4.320  2.330   
14             3  2015-02-14   311.0        16.20   0.846   4.000  1.060   
15             3  2013-03-14    91.3        42.80   0.262   1.395  0.425   
16             4  2013-04-24   247.0         7.30   0.755   1.349  0.139   
17             5  2017-01-24   249.0        46.00   2.300  15.850  0.000   
18             5  2016-12-08   240.0       767.00   0.137  27.250  0.050   
19             5  2016-05-25    89.5        13.20   2.920  14.542  0.142   
20             5  2013-03-09   132.0        53.00   2.530   0.350  0.050   
21             6  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
22             7  2014-06-03     5.0        18.30   0.028   0.703  0.133   
23             8  2013-03-07   322.0      1410.00   1.330   2.532  0.192   
24             9  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
25            10  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
26            11  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
27            12  2013-03-22    62.5        34.40   0.451   2.647  0.377   
28            13  2017-04-28   280.0       136.00   1.800  21.100  1.500   
29            13  2017-02-06    80.0        73.00   1.200   4.900  0.720   
..           ...         ...     ...          ...     ...     ...    ...   
72           103  2017-02-11     NaN        29.00   0.200   0.590    NaN   
73           103  2016-12-08    28.0        33.50   4.800   0.970  0.110   
74           103  2016-06-17    70.0        51.80   0.300   1.050  0.000   
75           103  2014-05-02     8.0        63.60   0.248   1.221  0.161   
76           103  2013-01-27   214.0       189.00   0.468   1.142  0.122   
77           104  2014-05-02    30.5        17.40   0.161   1.152  0.092   
78           104  2013-01-27   233.0       240.00   0.558   1.243  0.203   
79           105  2017-02-11   910.0       720.00   1.500   1.650  1.120   
80           105  2013-04-14   303.0        16.00   0.874   1.887  0.097   
81           106  2017-02-06   680.0      4000.00   1.200   5.700  3.680   
82           106  2016-12-01   272.0       801.00   0.800   0.700  0.120   
83           106  2016-06-17   151.0         8.15   0.096   0.680  0.000   
84           106  2016-05-25   157.0       137.00   1.600   1.640  0.000   
85           106  2015-02-03   155.0         8.20   0.379   1.296  0.085   
86           106  2013-03-09    10.5        17.00   0.124   0.856  0.106   
87           107  2017-04-19     NaN        47.00   0.480   1.680  0.570   
88           107  2017-01-24   124.0        90.00   0.570   2.190  0.540   
89           107  2016-12-04     7.0         7.20  10.300   0.950  0.200   
90           107  2016-05-14    52.0        31.10   0.281   2.489  0.309   
91           107  2015-02-20    16.0        31.70   0.227   1.412  0.342   
92           107  2013-03-24     6.0         7.80   0

In [12]:
#############################################################################################################
#                               Estimate Exceedances of Faclility Effluent Limits
#                      
#############################################################################################################     
def CalcExceedances(pd_Concs, pollLS):
    '''
    #for each facility in database, calculate exceedance for each pollutant constituent in pollLS list
    #do the Exceedance Calculation = max(0,(Constituent Concentration - NEL))
    # if no exceedance, then report 0. report NaN sample result is NaN
    #INPUT:
        pd_Concs: dataframe of concentrations [Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax]
        pollLS: list of pollutant constituents we want to analyze (constituent list needs to match those in pd_Concs and FacsNELs dataframes)
    #Return dataframe [Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax]
    '''
    pd_FacExceedances = pd_Concs   
    #     calculate exceedances:
    for Constituent in pollLS:
        if Constituent != 'phmin': #exc = concentration - nel
            pd_FacExceedances['exc_' + Constituent] = pd_FacExceedances['c_' + Constituent] - pd_FacExceedances['nel_' + Constituent]
        else: #phmin: exc = nel - phmin
            pd_FacExceedances['exc_' + Constituent] = pd_FacExceedances['nel_' + Constituent] - pd_FacExceedances['c_' + Constituent]          
        #replace <0 values w/ 0 (meaning no exceedance)
        pd_FacExceedances.loc[pd_FacExceedances['exc_' + Constituent]<0, 'exc_' + Constituent] = 0 
    return pd_FacExceedances

#############################################################################################################
#                          
#                                (DEFINE GLOBAL VARIABLE: pd_exFacExceedances)
############################################################################################################# 
start_time = time.time()
pd_exFacExceedances = CalcExceedances (pd_exMaxConcs, pollLS)
print('Concentrations in excess of wet/dry season NELs')
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
display(pd_exFacExceedances)


print (pollLS)
# print (pd_exMaxConcs.columns.values)
# print (pd_exFacExceedances.columns.values)

Concentrations in excess of wet/dry season NELs
--- 0.1509997844696045 execution time in seconds ---


Facility_ID sample_date   c_tss  c_turbidity     c_p     c_n   c_nn  \
0              1  2017-04-19   122.0          NaN     NaN   2.430    NaN   
1              1  2017-02-11    59.0        13.00   0.097   0.580    NaN   
2              1  2016-12-04    80.0        71.20   0.300   0.910  0.120   
3              1  2016-06-17    83.0        81.40   0.250   0.940  0.200   
4              1  2015-02-20    33.5        17.50   0.176   1.830  0.190   
5              1  2014-04-13    14.0         8.50   0.244   2.037  0.247   
6              1  2013-03-09   163.0        24.40   0.155   1.199  0.239   
7              2  2017-04-20     NaN        13.00     NaN   0.780    NaN   
8              2  2017-01-21     0.0        31.00   0.100   1.580  0.000   
9              2  2016-05-05     7.0         4.90   0.066   2.672  0.212   
10             2  2014-06-03    18.5         3.30   0.075   1.136  0.116   
11             2  2013-03-09    80.0         7.80   0.167   0.761  0.061   
12             3  2017-04-28     NaN          NaN     NaN     NaN    NaN   
13             3  2016-12-01   172.0       277.00   1.100   4.320  2.330   
14             3  2015-02-14   311.0        16.20   0.846   4.000  1.060   
15             3  2013-03-14    91.3        42.80   0.262   1.395  0.425   
16             4  2013-04-24   247.0         7.30   0.755   1.349  0.139   
17             5  2017-01-24   249.0        46.00   2.300  15.850  0.000   
18             5  2016-12-08   240.0       767.00   0.137  27.250  0.050   
19             5  2016-05-25    89.5        13.20   2.920  14.542  0.142   
20             5  2013-03-09   132.0        53.00   2.530   0.350  0.050   
21             6  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
22             7  2014-06-03     5.0        18.30   0.028   0.703  0.133   
23             8  2013-03-07   322.0      1410.00   1.330   2.532  0.192   
24             9  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
25            10  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
26            11  2016-12-31  2010.0      4181.00  10.300  27.250  3.680   
27            12  2013-03-22    62.5        34.40   0.451   2.647  0.377   
28            13  2017-04-28   280.0       136.00   1.800  21.100  1.500   
29            13  2017-02-06    80.0        73.00   1.200   4.900  0.720   
..           ...         ...     ...          ...     ...     ...    ...   
72           103  2017-02-11     NaN        29.00   0.200   0.590    NaN   
73           103  2016-12-08    28.0        33.50   4.800   0.970  0.110   
74           103  2016-06-17    70.0        51.80   0.300   1.050  0.000   
75           103  2014-05-02     8.0        63.60   0.248   1.221  0.161   
76           103  2013-01-27   214.0       189.00   0.468   1.142  0.122   
77           104  2014-05-02    30.5        17.40   0.161   1.152  0.092   
78           104  2013-01-27   233.0       240.00   0.558   1.243  0.203   
79           105  2017-02-11   910.0       720.00   1.500   1.650  1.120   
80           105  2013-04-14   303.0        16.00   0.874   1.887  0.097   
81           106  2017-02-06   680.0      4000.00   1.200   5.700  3.680   
82           106  2016-12-01   272.0       801.00   0.800   0.700  0.120   
83           106  2016-06-17   151.0         8.15   0.096   0.680  0.000   
84           106  2016-05-25   157.0       137.00   1.600   1.640  0.000   
85           106  2015-02-03   155.0         8.20   0.379   1.296  0.085   
86           106  2013-03-09    10.5        17.00   0.124   0.856  0.106   
87           107  2017-04-19     NaN        47.00   0.480   1.680  0.570   
88           107  2017-01-24   124.0        90.00   0.570   2.190  0.540   
89           107  2016-12-04     7.0         7.20  10.300   0.950  0.200   
90           107  2016-05-14    52.0        31.10   0.281   2.489  0.309   
91           107  2015-02-20    16.0        31.70   0.227   1.412  0.342   
92           107  2013-03-24     6.0         7.80   0

['tss', 'turbidity', 'p', 'n', 'nn', 'an', 'og', 'cu', 'zn', 'fe', 'phmin', 'phmax']


In [13]:
#############################################################################################################
#       CALCULATE EXISTING AGE FACTOR WEIGHTED AVERAGE FACILITY EXCEEDANCE VALUES FOR EACH CONSTITUENT:
#       
#############################################################################################################   
'''
Age factor acknowledges fact that more recent samples are a better representation of facility pollutant discharge 
(i.e. sampling data) and housekeeping-operations (i.e. inspections) realities. But, historic data as a whole also tells part 
of story (i.e. we want to dampen whipsaw effects that may occur if we only considered most recent data).

AF = exp(-SampleRank)
SampleRank = Newest sample = 1
              Second Newest sample = 2
              ...
              nth Newest Sample = n (out of n samples)
'''
def AFWFacExceedances(pd_FacExceedances, pollLS):
    '''
    CALCULATE AGE FACTOR WEIGHTED AVERAGE FOR EACH CONSTITUENT:

    Age factor acknowledges fact that more recent samples are a better representation of facility pollutant discharge
    (i.e. sampling data) and housekeeping-operations (i.e. inspections) realities. But, historic data as a whole also tells part
    of story (i.e. we want to dampen whipsaw effects that may occur if we only considered most recent data).

    AF = exp(-SampleRank)
    SampleRank = Newest sample = 1
                  Second Newest sample = 2
                  ...
                  nth Newest Sample = n (out of n samples)

    INPUTS:
        pd_FacExceedances: dataframe holding exceedances
            FORMAT: ExPollConc.id, Facility_ID, Sample_Date, exceedance concentrations
        pollLS: list of polluant constituents that can be found in the dataframe's exceedance concentrations
        ShowCalculations: True if you want output of calculation summary. false if not

    RETURN:
        DataFrame of age factor weighted averages.
        FORMAT: Facility_ID, AFwtd_c_conc...
    '''
    #calculate age factor weighted averages for each constituent in pollLS FOR each facility IN DATABASE.
    #write these averages into a dataframe called pd_AFWFacExceedances [Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax]
    pd_AFWFacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
    #insert blank columns:
    for Constituent in pollLS:
        #CALC AGE FACTOR
        pd_FacExceedances['c_' + Constituent + '_AF'] =np.exp(-pd_FacExceedances['c_' + Constituent + '_SR'])
        #CALC AGE FACTOR WTD CONCENTRATION        
        pd_FacExceedances['c_' + Constituent + '_AF*c'] = pd_FacExceedances[
            'c_' + Constituent + '_AF'] * pd_FacExceedances[
                'exc_' + Constituent]
    #sum AF and AF*c columns (just do all the columns in pd_FacExceedances for now. make more efficient if need to)
    pd_sums = pd_FacExceedances.groupby(['Facility_ID']).sum() 
#     #setup pd_AFWExceedances to include summed data
    #and do wtd average:
    for Constituent in pollLS:
        pd_AFWFacExceedances['c_' + Constituent + '_AFWtd'] =  pd_sums['c_' + Constituent + '_AF*c']/pd_sums['c_' + Constituent + '_AF']
    pd_AFWFacExceedances.reset_index(inplace = True)
    return pd_AFWFacExceedances

#############################################################################################################
#                        calculate age factor exceedances of existing samples in pd_exFacExceedances
#                                    DEFINE GLOBAL VARIABLE: pd_exAFWFacExceedances
############################################################################################################# 
start_time = time.time()
pd_exAFWFacExceedances = AFWFacExceedances(pd_exFacExceedances, pollLS)
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
print ('Age Factor Weighted Averages:')
display(pd_exAFWFacExceedances)

--- 0.07000017166137695 execution time in seconds ---
Age Factor Weighted Averages:


Facility_ID  c_tss_AFWtd  c_turbidity_AFWtd  c_p_AFWtd  c_n_AFWtd  \
0             1    52.064543          19.729711   0.067223   1.296165   
1             2     0.961758           3.745947   0.004876   0.637805   
2             3          NaN                NaN        NaN        NaN   
3             4   197.000000           0.000000   0.655000   0.829000   
4             5   180.960566         199.986459   1.752511  17.437221   
5             6  1960.000000        4166.000000  10.200000  26.730000   
6             7     0.000000          12.800000   0.000000   0.323000   
7             8          NaN                NaN        NaN        NaN   
8             9          NaN        4180.500000  10.270000  27.070000   
9            10  1960.000000        4166.000000  10.200000  26.730000   
10           11  1960.000000        4166.000000  10.200000  26.730000   
11           12    12.500000          19.400000   0.351000   2.127000   
12           13   336.807197         469.755775   1.674704  15.243243   
13           14  1960.000000        4166.000000  10.200000  26.730000   
14           15  1960.000000        4166.000000  10.200000  26.730000   
15           34  1960.000000        4166.000000  10.200000  26.730000   
16           41          NaN        4173.000000  10.170000  26.700000   
17           78    72.443042          99.914697   0.467443   1.707622   
18           79          NaN                NaN        NaN        NaN   
19           80          NaN          13.000000   0.148000   0.940000   
20           81  1960.000000        4166.000000  10.200000  26.730000   
21           82          NaN                NaN        NaN        NaN   
22           89     0.000000           0.000000   0.000000   0.496000   
23           90          NaN        4178.000000  10.250000  26.900000   
24           91          NaN                NaN        NaN        NaN   
25           92          NaN        4179.750000  10.260000  27.000000   
26           93          NaN        4178.000000  10.250000  26.900000   
27           94    43.568510          30.576732   1.132181   1.131914   
28           95  1960.000000        4166.000000  10.200000  26.730000   
29           96    16.559240           4.868334   0.504841   4.083533   
30           97     0.000000           0.000000   0.258000   2.260000   
31           98          NaN                NaN        NaN        NaN   
32           99          NaN        4178.000000  10.250000  26.900000   
33          100     1.201509           7.864125   0.152872   2.174979   
34          101          NaN                NaN        NaN        NaN   
35          102          NaN        4178.000000  10.250000  26.900000   
36          103    14.732924          16.599986   0.691190   0.829885   
37          104    49.581809          69.211417   0.197012   0.758822   
38          105   696.752557         515.665239   1.231643   1.193739   
39          106          NaN                NaN        NaN        NaN   
40          107          NaN          49.764373   1.282323   1.405886   
41          108          NaN                NaN        NaN        NaN   
42          109          NaN          15.400000   0.860000   3.290000   
43          110  1960.000000        4166.000000  10.200000  26.730000   
44          112    28.238849          23.715705   0.198266   0.706055   
45          240          NaN                NaN        NaN        NaN   

    c_nn_AFWtd  c_an_AFWtd  c_og_AFWtd  c_cu_AFWtd  c_zn_AFWtd    c_fe_AFWtd  \
0     0.029425         NaN    0.000000         NaN         NaN           NaN   
1     0.031165         NaN    0.000000         NaN         NaN           NaN   
2          NaN         NaN    0.000000         NaN         NaN           NaN   
3     0.000000         NaN    0.000000         NaN         NaN           NaN   
4     0.004532         NaN    1.421297         NaN         NaN           NaN   
5     3.500000         NaN   44.000000         NaN         NaN           NaN   
6     0.0430

In [14]:
#############################################################################################################
#                       Estimate Facility Runoff Volumes
#                       DEFINE GLOBAL VARIABLE: pd_RunoffVols
#############################################################################################################   
#get facility imperviousness and area. order by Facility_ID so it's given in same order as monthly rain data dataframe
q_facDat = session.query(Facility_Chars.id.label('Facility_ID'), 
                         Facility_Chars.Indus_Area, 
                         Facility_Chars.Imperv.label('Imperv')).order_by('Facility_ID')
pd_facDat = pd.read_sql(q_facDat.statement,session.bind)

#get monthly rain data for each facility. order by facility_id so order matches facility data dataframe
q_rain = session.query(Facility_Chars.id.label('Facility_ID'), Facility_Monthly_Rain).filter(
    Facility_Chars.facility_monthly_rain_id == Facility_Monthly_Rain.id).order_by('Facility_ID')
pd_rainDat = pd.read_sql(q_rain.statement,session.bind)

#create a new dataframe to hold rain volumes
pd_RunoffVols = pd_facDat.loc[:,['Facility_ID']] #put facilities into the new dataframe
#now calculate volumes for each month:
for mo in range(1,13):
    pd_RunoffVols[calendar.month_name[mo]] = pd.DataFrame(pd_facDat['Indus_Area'] * pd_facDat['Imperv'] * pd_rainDat[calendar.month_name[mo]]/12)
#add monthlys together to get annual volume
pd_RunoffVols['Annual_Volume'] = pd_RunoffVols[[calendar.month_name[mo] for mo in range (1,13)]].sum(axis = 1)
display(pd_RunoffVols)

Facility_ID        January       February          March          April  \
0              1  295325.500000  335399.166667  352822.500000  250024.833333   
1              2  338916.586546  246040.406579  241152.186581  101837.916631   
2              3   77993.831554   75165.810944   79184.577074   53434.705206   
3              4   21601.669993   18133.109161   16552.499161    9439.754164   
4              5   50545.681653   57766.493317   56824.648318   36261.032490   
5              6   15974.190828   18427.906661   18127.451661   11667.669163   
6              7   10058.130000    7407.150000    7069.280000    2858.900000   
7              8   31502.679978   29712.754979   32397.642477   21658.092485   
8              9    9823.037518    8034.365015    6568.240012    3782.602507   
9             10    7359.825004    4947.100003    3973.900002    2108.600001   
10            11   38203.193333   36076.720000   44289.306667   30210.586667   
11            12   15778.000002   11261.833335   11004.583335    6116.833334   
12            13   10628.624992    8401.674994    9758.089992    4534.879997   
13            14    5740.670832    3474.224166    3354.937499    1520.905000   
14            15    4947.973337    3860.506670    3806.133336    1644.793335   
15            16    9943.041593    7764.165194    9544.092393    5892.172796   
16            17    3686.471831    2784.256357    2629.036490    1096.240308   
17            18   53844.879604   33653.049752   40716.035503   21022.769352   
18            19    3815.469177    2877.239051    4020.986252    2528.753576   
19            20    8321.346913    6368.778897    8832.421763    7757.854130   
20            21    5387.240724    5977.084599    5387.240724    3788.108441   
21            22   39352.347381   24823.730133   20301.577744   12892.945108   
22            23   52497.744370   48245.590621   42685.081871   20115.958123   
23            24    8061.145988    8891.241112   10108.713962    8983.473904   
24            25   11225.985481   10110.989181   14292.225308    8261.108955   
25            26    9456.558647    7778.121831    9006.246330    5874.528856   
26            27   13384.363069    7531.143068   10457.753068    5306.919467   
27            28    7193.151073    8000.020048    9098.735248    7897.015498   
28            29    7272.970203    5877.276977    7580.278803    6504.698702   
29            30    7089.868296    5606.305547    5653.154897    3279.454498   
..           ...            ...            ...            ...            ...   
210          211            NaN            NaN            NaN            NaN   
211          212            NaN            NaN            NaN            NaN   
212          213            NaN            NaN            NaN            NaN   
213          214            NaN            NaN            NaN            NaN   
214          215            NaN            NaN            NaN            NaN   
215          216            NaN            NaN            NaN            NaN   
216          217            NaN            NaN            NaN            NaN   
217          218            NaN            NaN            NaN            NaN   
218          219            NaN            NaN            NaN            NaN   
219          220            NaN            NaN            NaN            NaN   
220          221            NaN            NaN            NaN            NaN   
221          222            NaN            NaN            NaN            NaN   
222          223            NaN            NaN            NaN            NaN   
223          224            NaN            NaN            NaN            NaN   
224          225            NaN            NaN            NaN            NaN   
225          226            NaN            NaN            NaN            NaN   
226          227            NaN            NaN            NaN            NaN   
227          228            NaN            NaN            NaN            NaN   
228  

In [15]:
#############################################################################################################
#                       Calculate raw pollutant exceedance potential scores (PEP_raw)
#                         PEP_raw = AFWtd Exceedance * Annual Runoff Volume (cu. ft)
############################################################################################################   
def _HELPER_calc_PEP_raw(row, Constituent, pd_RunoffVols):
    #HELPER function to calculate PEP_raw
    AnnRunoffVol = pd_RunoffVols.loc[pd_RunoffVols['Facility_ID']==row.loc['Facility_ID'],'Annual_Volume'].values[0]
    AFWFacExceedVal = row.loc['c_' + Constituent + '_AFWtd']
    return  AFWFacExceedVal * AnnRunoffVol
def CalcPEP_Raw(pd_AFWFacExceedances,pollLS, pd_RunoffVols):
    #use age factor weighted scores to calculate raw PEP scores for each constituent pollutant
    #input: 
        #pd_AFWFacExceedances: [Facility_ID	AFWtd_c_tss	AFWtd_c_turbidity	AFWtd_c_p	AFWtd_c_n	AFWtd_c_nn	AFWtd_c_an	AFWtd_c_og	AFWtd_c_cu	AFWtd_c_zn	AFWtd_c_fe	AFWtd_c_phmin	AFWtd_c_phmax]
        #pollLS: pollutant constituent list
        #pd_RunoffVols: RUNOFF VOLUMES [Facility_ID	January	February	March	April	May	June	July	August	September	October	November	December	Annual_Volume] 
    #output: pd_PEP_raw[	Facility_ID	PEP_raw_tss	PEP_raw_turbidity	PEP_raw_p	PEP_raw_n	PEP_raw_nn	PEP_raw_an	PEP_raw_og	PEP_raw_cu	PEP_raw_zn	PEP_raw_fe	PEP_raw_phmin	PEP_raw_phmax]

    #initialize pd_PEP_raw dataframe w/ Facility_IDs from pd_AFWFacExceedances
    pd_PEP_raw = pd_AFWFacExceedances.loc[:,['Facility_ID']]
#     display(pd_PEP_raw)
#     pd_PEP_raw.reset_index(drop=True)
    #for each facility in pd_exPEP_raw, calculate PEP_Raw SCORE for each pollutant constituent in the pollLS LIST:
    for Constituent in pollLS:
        pd_PEP_raw['PEP_raw_' + Constituent] = pd_AFWFacExceedances.apply(lambda row: 
                                               _HELPER_calc_PEP_raw(row,Constituent, pd_RunoffVols), axis = 1)    
    return pd_PEP_raw

#############################################################################################################
#                              calculate existing PEP_raw scores
#                              DEFINE GLOBAL VARIABLE: pd_exPEP_raw
#############################################################################################################  

pd_exPEP_raw = CalcPEP_Raw(pd_exAFWFacExceedances,pollLS,pd_RunoffVols)
display(pd_exPEP_raw)

Facility_ID   PEP_raw_tss  PEP_raw_turbidity     PEP_raw_p     PEP_raw_n  \
0             1  1.546670e+08       5.861063e+07  1.996976e+05  3.850488e+06   
1             2  2.297361e+06       8.947981e+06  1.164828e+04  1.523531e+06   
2             3           NaN                NaN           NaN           NaN   
3             4  3.033362e+07       0.000000e+00  1.008554e+05  1.276476e+05   
4             5  8.766133e+07       9.687790e+07  8.489555e+05  8.446978e+06   
5             6  3.024941e+08       6.429543e+08  1.574204e+06  4.125340e+06   
6             7  0.000000e+00       8.912283e+05  0.000000e+00  2.248959e+04   
7             8           NaN                NaN           NaN           NaN   
8             9           NaN       2.658819e+08  6.531772e+05  1.721666e+06   
9            10  8.516068e+07       1.810099e+08  4.431831e+05  1.161400e+06   
10           11  7.296678e+08       1.550916e+09  3.797251e+06  9.951031e+06   
11           12  1.386649e+06       2.152079e+06  3.893710e+04  2.359522e+05   
12           13  2.559726e+07       3.570131e+07  1.272771e+05  1.158482e+06   
13           14  6.666276e+07       1.416924e+08  3.469184e+05  9.091304e+05   
14           15  6.239775e+07       1.326271e+08  3.247230e+05  8.509652e+05   
15           34  5.638667e+08       1.198504e+09  2.934408e+06  7.689876e+06   
16           41           NaN       2.750512e+09  6.703262e+06  1.759853e+07   
17           78  0.000000e+00       0.000000e+00  0.000000e+00  0.000000e+00   
18           79           NaN                NaN           NaN           NaN   
19           80           NaN       1.620933e+06  1.845370e+04  1.172060e+05   
20           81  2.181791e+08       4.637419e+08  1.135422e+06  2.975473e+06   
21           82           NaN                NaN           NaN           NaN   
22           89  0.000000e+00       0.000000e+00  0.000000e+00  2.248371e+05   
23           90           NaN       2.968967e+09  7.283846e+06  1.911565e+07   
24           91           NaN                NaN           NaN           NaN   
25           92           NaN       1.526735e+09  3.747665e+06  9.862276e+06   
26           93           NaN       6.019064e+08  1.476673e+06  3.875367e+06   
27           94  1.793600e+07       1.258763e+07  4.660887e+05  4.659791e+05   
28           95  1.392570e+09       2.959922e+09  7.247048e+06  1.899153e+07   
29           96  2.421862e+06       7.120152e+05  7.383521e+04  5.972347e+05   
30           97  0.000000e+00       0.000000e+00  4.986590e+04  4.368098e+05   
31           98           NaN                NaN           NaN           NaN   
32           99           NaN       1.778818e+09  4.364023e+06  1.145290e+07   
33          100  1.704068e+06       1.115348e+07  2.168143e+05  3.084714e+06   
34          101           NaN                NaN           NaN           NaN   
35          102           NaN       4.912121e+08  1.205104e+06  3.162663e+06   
36          103  1.620177e+07       1.825497e+07  7.600998e+05  9.126226e+05   
37          104  2.139957e+07       2.987173e+07  8.503066e+04  3.275085e+05   
38          105  1.350819e+09       9.997382e+08  2.387829e+06  2.314344e+06   
39          106           NaN                NaN           NaN           NaN   
40          107           NaN       1.259836e+08  3.246333e+06  3.559146e+06   
41          108           NaN                NaN           NaN           NaN   
42          109           NaN       1.086004e+07  6.064696e+05  2.320099e+06   
43          110  0.000000e+00       0.000000e+00  0.000000e+00  0.000000e+00   
44          112  0.000000e+00       0.000000e+00  0.000000e+00  0.000000e+00   
45          240           NaN                NaN           NaN           NaN   

      PEP_raw_nn    PEP_raw_an    PEP_raw_og    PEP_raw_cu    PEP_raw_zn  \
0   8.741303e+04           NaN  0.000000e+00           NaN           NaN   
1   7.444524e+04           NaN  0.000000e+00           NaN           NaN   
2            Na

In [16]:
#############################################################################################################
#                       Calculate normalized pollutant exceedance potential scores (PEP_norm)
#                         PEP_Norm = (PEP_raw - PEPmin) / (PEPMax - PEPmin)
############################################################################################################   
'''
NORMALIZE the raw Pollutant Exceedance Potential scores held in a pd_PEP_raw dataframe to a new dataframe called pd_PEP_norm.
Use calculation:
PEP_Norm = (PEP_raw - PEPmin) / (PEPMax - PEPmin)

Hold the PEPmax and PEPmin baseline scores used for the normalization in a dataframe called pd_NormBaselinePEP
****NOTE: LATER, we'll need to write the norm. basis to file
          This will allow us to use a common baseline in future (when we get more data, we'll want to have same baseline)         
'''
#############################################################################################################
#                           BUILD BASELINE dataframe pd_NormBaselinePEP
#                              DEFINE GLOBAL VARIABLE: pd_NormBaselinePEP
############################################################################################################  
##Use pd_exPEP_Ras data as our baseline max. Use 0 as min for all:
dict_NormBaselinePEP = {'PEP_Baseline_' + Constituent: [pd_exPEP_raw.loc[:,'PEP_raw_' + Constituent].max(),
                                             0]
                                             for Constituent in pollLS}
dict_NormBaselinePEP['MaxMin'] = ['Max','Min'] #add column identifying if row is max or min
pd_NormBaselinePEP = pd.DataFrame(dict_NormBaselinePEP) #write dict to new dataframe 
print ('This is the pd_NormBaselinePEP dataframe:')
display(pd_NormBaselinePEP)

#############################################################################################################
#                                        CALCULATE PEP_norm
#                         
############################################################################################################ 
def CalcPEP_norm(pd_PEP_raw,pollLS, pd_NormBaselinePEP):
    #calculate PEP_norm for each constituent pollutant of each facility in pd_PEP_raw
    #return pd_PEP_norm [	Facility_ID	PEP_norm_tss	PEP_norm_turbidity	PEP_norm_p	PEP_norm_n	PEP_norm_nn	PEP_norm_an	PEP_norm_og	PEP_norm_cu	PEP_norm_zn	PEP_norm_fe	PEP_norm_phmin	PEP_norm_phmax]
    
    #initialize pd_PEP_norm dataframe w/ Facility_IDs from pd_exPEP_raw
    pd_PEP_norm = pd_PEP_raw.loc[:,['Facility_ID']]
    for Constituent in pollLS:
        BLmax= pd_NormBaselinePEP.loc[pd_NormBaselinePEP['MaxMin']=='Max', 'PEP_Baseline_' + Constituent].values[0] 
        BLmin= pd_NormBaselinePEP.loc[pd_NormBaselinePEP['MaxMin']=='Min', 'PEP_Baseline_' + Constituent].values[0]        
        pd_PEP_norm['PEP_norm_' + Constituent] = (pd_PEP_raw['PEP_raw_' + Constituent] - BLmin) / (BLmax - BLmin)
    return pd_PEP_norm

#############################################################################################################
#                       Normalize existing raw pollutant exceedance potential scores 
#                         (DEFINE GLOBAL VARIABLE: pd_exPEP_norm)
############################################################################################################   
print('This is the pd_exPEP_norm dataframe:')
pd_exPEP_norm = CalcPEP_norm(pd_exPEP_raw,pollLS,pd_NormBaselinePEP)
display(pd_exPEP_norm)

#TO DO:  WRITE existing NORMALIZED PEP SCORES TO DB: 

This is the pd_NormBaselinePEP dataframe:


MaxMin  PEP_Baseline_an  PEP_Baseline_cu  PEP_Baseline_fe  PEP_Baseline_n  \
0    Max     7.452262e+06     2.297250e+08     4.448060e+09    1.911565e+07   
1    Min     0.000000e+00     0.000000e+00     0.000000e+00    0.000000e+00   

   PEP_Baseline_nn  PEP_Baseline_og  PEP_Baseline_p  PEP_Baseline_phmax  \
0     2.600866e+06     3.126724e+07    7.283846e+06       497346.455645   
1     0.000000e+00     0.000000e+00    0.000000e+00            0.000000   

   PEP_Baseline_phmin  PEP_Baseline_tss  PEP_Baseline_turbidity  \
0       703512.890691      1.392570e+09            2.968967e+09   
1            0.000000      0.000000e+00            0.000000e+00   

   PEP_Baseline_zn  
0     1.051438e+09  
1     0.000000e+00

This is the pd_exPEP_norm dataframe:


Facility_ID  PEP_norm_tss  PEP_norm_turbidity  PEP_norm_p  PEP_norm_n  \
0             1      0.111066            0.019741    0.027417    0.201431   
1             2      0.001650            0.003014    0.001599    0.079701   
2             3           NaN                 NaN         NaN         NaN   
3             4      0.021782            0.000000    0.013846    0.006678   
4             5      0.062949            0.032630    0.116553    0.441888   
5             6      0.217220            0.216558    0.216123    0.215810   
6             7      0.000000            0.000300    0.000000    0.001177   
7             8           NaN                 NaN         NaN         NaN   
8             9           NaN            0.089554    0.089675    0.090066   
9            10      0.061154            0.060967    0.060845    0.060757   
10           11      0.523972            0.522376    0.521325    0.520570   
11           12      0.000996            0.000725    0.005346    0.012343   
12           13      0.018381            0.012025    0.017474    0.060604   
13           14      0.047870            0.047724    0.047628    0.047559   
14           15      0.044808            0.044671    0.044581    0.044517   
15           34      0.404911            0.403677    0.402865    0.402282   
16           41           NaN            0.926421    0.920292    0.920635   
17           78      0.000000            0.000000    0.000000    0.000000   
18           79           NaN                 NaN         NaN         NaN   
19           80           NaN            0.000546    0.002534    0.006131   
20           81      0.156674            0.156196    0.155882    0.155656   
21           82           NaN                 NaN         NaN         NaN   
22           89      0.000000            0.000000    0.000000    0.011762   
23           90           NaN            1.000000    1.000000    1.000000   
24           91           NaN                 NaN         NaN         NaN   
25           92           NaN            0.514231    0.514517    0.515927   
26           93           NaN            0.202733    0.202733    0.202733   
27           94      0.012880            0.004240    0.063989    0.024377   
28           95      1.000000            0.996954    0.994948    0.993507   
29           96      0.001739            0.000240    0.010137    0.031243   
30           97      0.000000            0.000000    0.006846    0.022851   
31           98           NaN                 NaN         NaN         NaN   
32           99           NaN            0.599137    0.599137    0.599137   
33          100      0.001224            0.003757    0.029766    0.161371   
34          101           NaN                 NaN         NaN         NaN   
35          102           NaN            0.165449    0.165449    0.165449   
36          103      0.011634            0.006149    0.104354    0.047742   
37          104      0.015367            0.010061    0.011674    0.017133   
38          105      0.970018            0.336729    0.327825    0.121071   
39          106           NaN                 NaN         NaN         NaN   
40          107           NaN            0.042434    0.445689    0.186190   
41          108           NaN                 NaN         NaN         NaN   
42          109           NaN            0.003658    0.083262    0.121372   
43          110      0.000000            0.000000    0.000000    0.000000   
44          112      0.000000            0.000000    0.000000    0.000000   
45          240           NaN                 NaN         NaN         NaN   

    PEP_norm_nn  PEP_norm_an  PEP_norm_og  PEP_norm_cu  PEP_norm_zn  \
0      0.033609          NaN     0.000000          NaN          NaN   
1      0.028623          NaN     0.000000          NaN          NaN   
2           NaN          NaN     0.000000          NaN          NaN   
3      0.000000          NaN     0.000000          NaN          NaN   
4      0.000844          NaN 

In [17]:
#############################################################################################################
#                                        Sum Normalized PEP Scores
#                                write scores to new dataframe called pd_PEP_sum
############################################################################################################ 
def SumNormPEPs (pd_PEP_norm):
    #general function to sum normalized PEPs
    #CAUTION!!! REVISES passed dataframe to include summing column
    pd_PEP_norm.set_index('Facility_ID', inplace=True) #move FAcility ID to index temporarily
#     display(pd_PEP_norm['PEP_norm_sum'])
    pd_PEP_norm['PEP_norm_sum'] = pd_PEP_norm.sum(axis = 1) #sum norm scores for each facility
    pd_PEP_norm.reset_index(inplace=True) #move facility ID from index
    return(pd_PEP_norm) #return passed dataframe

#############################################################################################################
#                                  Sum existing Normalized PEP Scores
#                              (REDEFINE GLOBAL VARIABLE: pd_exPEP_norm)
############################################################################################################ 

# display(pd_exPEP_norm.loc[:,['Facility_ID', 'PEP_norm_sum']])
pd_exPEP_norm = SumNormPEPs(pd_exPEP_norm) #revise pd_exPEP_Norm to include summing column
display(pd_exPEP_norm)

Facility_ID  PEP_norm_tss  PEP_norm_turbidity  PEP_norm_p  PEP_norm_n  \
0             1      0.111066            0.019741    0.027417    0.201431   
1             2      0.001650            0.003014    0.001599    0.079701   
2             3           NaN                 NaN         NaN         NaN   
3             4      0.021782            0.000000    0.013846    0.006678   
4             5      0.062949            0.032630    0.116553    0.441888   
5             6      0.217220            0.216558    0.216123    0.215810   
6             7      0.000000            0.000300    0.000000    0.001177   
7             8           NaN                 NaN         NaN         NaN   
8             9           NaN            0.089554    0.089675    0.090066   
9            10      0.061154            0.060967    0.060845    0.060757   
10           11      0.523972            0.522376    0.521325    0.520570   
11           12      0.000996            0.000725    0.005346    0.012343   
12           13      0.018381            0.012025    0.017474    0.060604   
13           14      0.047870            0.047724    0.047628    0.047559   
14           15      0.044808            0.044671    0.044581    0.044517   
15           34      0.404911            0.403677    0.402865    0.402282   
16           41           NaN            0.926421    0.920292    0.920635   
17           78      0.000000            0.000000    0.000000    0.000000   
18           79           NaN                 NaN         NaN         NaN   
19           80           NaN            0.000546    0.002534    0.006131   
20           81      0.156674            0.156196    0.155882    0.155656   
21           82           NaN                 NaN         NaN         NaN   
22           89      0.000000            0.000000    0.000000    0.011762   
23           90           NaN            1.000000    1.000000    1.000000   
24           91           NaN                 NaN         NaN         NaN   
25           92           NaN            0.514231    0.514517    0.515927   
26           93           NaN            0.202733    0.202733    0.202733   
27           94      0.012880            0.004240    0.063989    0.024377   
28           95      1.000000            0.996954    0.994948    0.993507   
29           96      0.001739            0.000240    0.010137    0.031243   
30           97      0.000000            0.000000    0.006846    0.022851   
31           98           NaN                 NaN         NaN         NaN   
32           99           NaN            0.599137    0.599137    0.599137   
33          100      0.001224            0.003757    0.029766    0.161371   
34          101           NaN                 NaN         NaN         NaN   
35          102           NaN            0.165449    0.165449    0.165449   
36          103      0.011634            0.006149    0.104354    0.047742   
37          104      0.015367            0.010061    0.011674    0.017133   
38          105      0.970018            0.336729    0.327825    0.121071   
39          106           NaN                 NaN         NaN         NaN   
40          107           NaN            0.042434    0.445689    0.186190   
41          108           NaN                 NaN         NaN         NaN   
42          109           NaN            0.003658    0.083262    0.121372   
43          110      0.000000            0.000000    0.000000    0.000000   
44          112      0.000000            0.000000    0.000000    0.000000   
45          240           NaN                 NaN         NaN         NaN   

    PEP_norm_nn  PEP_norm_an  PEP_norm_og  PEP_norm_cu  PEP_norm_zn  \
0      0.033609          NaN     0.000000          NaN          NaN   
1      0.028623          NaN     0.000000          NaN          NaN   
2           NaN          NaN     0.000000          NaN          NaN   
3      0.000000          NaN     0.000000          NaN          NaN   
4      0.000844          NaN 

In [18]:
'''
#############################################################################################################
#                               CALCULATE WRS PEP BASE SCORES
#                      WRS PEP BASE SCORE = NORM_PEP_SCORE*(SampleUncertainty + 1) 
############################################################################################################ 
'''
def _HELPER_PEPUncertainty(ls_id, dict_unc):
    '''determine the uncertainty level based on sample method
        (retrieve list of sample methods from ExPollConcs table for facilities in ls_id; assign uncertainty level using dict_unc)
       input: 
            ls_id: list of facility ids
            dict_unc: dictionary of uncertainty values for each sample method
        return: 
            pd_unc: dataframe [Facility_ID, UncertaintyValue]
    '''
    #get sample method for each facility in ls_id list
    q = session.query(ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_method.label('sample_method')).filter(
        ExPollConcs.facility_id.in_(ls_id)).distinct(ExPollConcs.facility_id).order_by(ExPollConcs.facility_id)
    pd_samplemethod = pd.read_sql(q.statement,session.bind)
    #use dict_unc to assign uncertainty value for each facility's sample method
    pd_samplemethod['Uncertainty_Value'] = pd_samplemethod['sample_method'].apply(lambda val: dict_unc[val])
    return pd_samplemethod
    
def CalcWRSPEPBaseScore(pd_PEP_sum, Use_PrevUncertaintyVals):
    #calculate wrs pep base score = NORM_PEP_SCORE*(SampleUncertainty + 1) 
    #input: pd_PEP_sum dataframe containing COLUMNS [Facility_ID, PEP_norm_sum]
    #       Use_PrevUncertaintyVals: TRUE if want to use uncertainty_values prev. retrieved and stored in pd_exWRSPEPBaseScore
    #                                FALSE if want to retrieve uncertainty vals
    #return: dataframe of WRS PEP Base Scores
    
    #make a list of Facility IDs in pd_PEP_sum
    ls_id = [np.asscalar(id) for id in pd_PEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    #make the pd_WRSPEPBaseScore dataframe:
    #write uncertainty information into pd_WRSPEPBaseScore
    if Use_PrevUncertaintyVals:
        pd_WRSPEPBaseScore = pd.DataFrame() #make new df
        pd_WRSPEPBaseScore['Facility_ID'] = pd_exWRSPEPBaseScore['Facility_ID']
        pd_WRSPEPBaseScore['Uncertainty_Value'] = pd_exWRSPEPBaseScore['Uncertainty_Value']
    else:
        pd_WRSPEPBaseScore = _HELPER_PEPUncertainty(ls_id, {'infield':0.25, 'sim_MaxType':1.0, 'sim_EMC':0.0})
    #copy in PEP_norm_sum values
    pd_WRSPEPBaseScore['PEP_norm_sum'] = pd_PEP_sum['PEP_norm_sum']
    #calculate PEP wrs and then write result into column
    pd_WRSPEPBaseScore['PEP_BaseRisk'] = pd_WRSPEPBaseScore['PEP_norm_sum'] * (pd_WRSPEPBaseScore['Uncertainty_Value'] + 1)
    return pd_WRSPEPBaseScore

#############################################################################################################
#                            CALCULATE existing WRS PEP BASE SCORES & (TO DO: WRITE SCORES TO database) 
#                              (DEFINE GLOBAL VARIABLE: pd_exWRSPEPBaseScore)
############################################################################################################ 
#calc WRS PEP Base Scores for existing normalized PEP sums (pd_exPEP_norm)
pd_exWRSPEPBaseScore = CalcWRSPEPBaseScore(pd_exPEP_norm, False)
display(pd_exWRSPEPBaseScore)

Facility_ID sample_method  Uncertainty_Value  PEP_norm_sum  PEP_BaseRisk
0             1       infield               0.25      0.950855      1.188568
1             2       infield               0.25      0.114587      0.143233
2             3       infield               0.25      0.165571      0.206963
3             4       infield               0.25      0.042307      0.052883
4             5       infield               0.25      0.676885      0.846106
5             6   sim_MaxType               1.00      1.507800      3.015601
6             7       infield               0.25      0.002628      0.003285
7             8       infield               0.25      0.039578      0.049472
8             9   sim_MaxType               1.00      0.694638      1.389276
9            10   sim_MaxType               1.00      0.424489      0.848977
10           11   sim_MaxType               1.00      3.637074      7.274148
11           12       infield               0.25      0.043425      0.054282
12           13       infield               0.25      0.139923      0.174903
13           14   sim_MaxType               1.00      0.332285      0.664569
14           15   sim_MaxType               1.00      0.311025      0.622051
15           34   sim_MaxType               1.00      2.810628      5.621256
16           41   sim_MaxType               1.00      6.284403     12.568806
17           78       infield               0.25      0.000000      0.000000
18           79       infield               0.25      1.000000      1.250000
19           80       infield               0.25      0.067449      0.084311
20           81   sim_MaxType               1.00      1.550503      3.101006
21           82       infield               0.25      0.000000      0.000000
22           89       infield               0.25      0.014551      0.018188
23           90   sim_MaxType               1.00      7.142882     14.285764
24           91       infield               0.25      0.000000      0.000000
25           92   sim_MaxType               1.00      3.986771      7.973541
26           93   sim_MaxType               1.00      1.448095      2.896191
27           94       infield               0.25      0.105486      0.131857
28           95   sim_MaxType               1.00      6.941351     13.882702
29           96       infield               0.25      0.043359      0.054199
30           97       infield               0.25      0.029697      0.037121
31           98   sim_MaxType               1.00      0.697796      1.395592
32           99   sim_MaxType               1.00      4.278920      8.557839
33          100       infield               0.25      0.251203      0.314004
34          101   sim_MaxType               1.00      0.246027      0.492054
35          102   sim_MaxType               1.00      1.181782      2.363563
36          103       infield               0.25      0.327798      0.409747
37          104       infield               0.25      0.055504      0.069380
38          105       infield               0.25      4.029826      5.037282
39          106       infield               0.25      1.791474      2.239343
40          107       infield               0.25      2.458516      3.073145
41          108       infield               0.25      0.385001      0.481251
42          109       infield               0.25      1.031768      1.289710
43          110   sim_MaxType               1.00      0.000000      0.000000
44          112       infield               0.25      0.000000      0.000000
45          240       infield               0.25      0.000000      0.000000

In [19]:
'''
#############################################################################################################
#                                    CALCULATE WRS BASE SCORES  
#
############################################################################################################ 
CALCULATE WRS BASE SCORE:
    TABLE 1 Facilities: WRS_BASE = WRS_INHERENT + WRS_CONTROLLABLE
        WRS_CONTROLLABLE = WRS_BMP + WRS_PEP
        WRS_BMP = WRS_HOUSEKEEPING + WRS_PCBMP
    TABLE 1A Facilities: 
    TABLE 2 & non-permitted:

    INPUTS:
        pd_wrsNonPEPScores
        pd_wrsPEPScores
'''
def GET_pd_FacRisks(ls_id):
    '''helper function that takes in list of facility_char ids and returns dataframe of:
        Facility_ID
        Inherent base risk
        housekeeping bmp base risk
        sw plan base risk
        bmp inspection deficiency rate (pc base risk)
    
        these items are obtained by querying database table: Facility Risks
    '''    
    q_facriskIDs =  session.query(Facility_Chars.existing_facility_risk_id).filter(Facility_Chars.id.in_(ls_id)) #for facilities in pd_exPEP_sum, get existing_facility_risk_id records
    #use q_facriskIDs as filter on Facility_Risks table to get associated wrs pollutant base id
    q_facrisks = session.query(
        Facility_Chars.id.label('Facility_ID'),Facility_Risks.Category_RiskFactor, Facility_Risks.Inherent_BaseRisk, Facility_Risks.HousekeepingBMP_BaseRisk, Facility_Risks.SWPlan_BaseRisk, Facility_Risks.BMPInspectionDeficiency_Rate).filter(
            Facility_Risks.id.in_(q_facriskIDs)).filter(
                Facility_Risks.id == Facility_Chars.existing_facility_risk_id).order_by(Facility_Chars.id)
    pd_facrisks = pd.read_sql(q_facrisks.statement, session.bind)
    return pd_facrisks

def CalcWRSBaseScore(pd_wrsNonPEPScores, pd_wrsPEPScores):
    '''
    CALCULATE WRS BASE SCORE:
        TABLE 1 Facilities: WRS_BASE = WRS_INHERENT + WRS_CONTROLLABLE
            WRS_CONTROLLABLE = WRS_BMP + WRS_PEP
            WRS_BMP = WRS_HOUSEKEEPING + WRS_PCBMP
        TABLE 1A Facilities: 
        TABLE 2 & non-permitted:

        INPUTS:
            pd_wrsNonPEPScores [Facility_ID	Category_RiskFactor	Inherent_BaseRisk	HousekeepingBMP_BaseRisk	SWPlan_BaseRisk	BMPInspectionDeficiency_Rate]
            pd_wrsPEPScores [	Facility_ID	sample_method	Uncertainty_Value	PEP_norm_sum	PEP_BaseRisk ]

    '''
    #initialize pd_exPEP_norm dataframe w/ Facility_IDs from pd_exPEP_raw
    pd_wrsBaseScores = pd.merge(pd_wrsNonPEPScores, pd_wrsPEPScores, on='Facility_ID')
    
    #calculate Table 1 scores (no need to differentiate tables now. all facilities are table 1)
    pd_wrsBaseScores['BMP_BaseRisk'] = pd_wrsBaseScores['HousekeepingBMP_BaseRisk'] + pd_wrsBaseScores['BMPInspectionDeficiency_Rate']
    pd_wrsBaseScores['Controllable_BaseRisk'] = pd_wrsBaseScores['BMP_BaseRisk'] + pd_wrsBaseScores['PEP_BaseRisk']
    #calculate total score:
    pd_wrsBaseScores['Total_BaseRisk'] = pd_wrsBaseScores['Inherent_BaseRisk'] + pd_wrsBaseScores['Controllable_BaseRisk']
    return pd_wrsBaseScores

#############################################################################################################
#                             CALCULATE existing WRS BASE SCORES 
#                       (DEFINE GLOBAL VARIABLE: pd_exwrsNonPEPScores & pd_exwrsBaseScores)
############################################################################################################ 
#make a list of Facility IDs in pd_exWRSPEPBaseScore
ls_id = [np.asscalar(id) for id in pd_exWRSPEPBaseScore['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
#get nonPEP WRS scores for each facility
pd_exwrsNonPEPScores = GET_pd_FacRisks(ls_id)
#make base scores using existing sub-scores.
pd_exwrsBaseScores =  CalcWRSBaseScore(pd_exwrsNonPEPScores,pd_exWRSPEPBaseScore.loc[:,['Facility_ID','PEP_BaseRisk']])
display(pd_exwrsBaseScores)

Facility_ID  Category_RiskFactor  Inherent_BaseRisk  \
0             1           150.003019           1.311222   
1             2           150.003019           1.292442   
2             3           150.003019           1.354261   
3             4           150.003019           1.054281   
4             5           150.003019           1.289362   
5             6           150.003019           1.287252   
6             7           150.003019           1.271192   
7             8           150.003019           1.350908   
8             9           150.003019           1.002867   
9            10           150.003019           1.000678   
10           11           150.003019           1.364567   
11           12           150.003019           1.568001   
12           13           150.003019           1.062798   
13           14           150.003019           1.255812   
14           15           150.003019           1.008215   
15           34           150.003019           1.257464   
16           41           150.003019           1.263079   
17           78           150.003019           1.069244   
18           79           150.003019           1.437338   
19           80           150.003019           1.283274   
20           81           150.003019           1.026441   
21           82           150.003019           1.435764   
22           89           150.003019           1.087887   
23           90           150.003019           1.379098   
24           91           150.003019           1.351401   
25           92           150.003019           1.004328   
26           93           150.003019           1.283793   
27           94           150.003019           1.365954   
28           95           150.003019           1.031105   
29           96           150.003019           1.273323   
30           97           150.003019           1.273480   
31           98           150.003019           1.575076   
32           99           150.003019           1.293695   
33          100           150.003019           1.092966   
34          101           150.003019           1.169269   
35          102           150.003019           1.014761   
36          103           150.003019           1.092080   
37          104           150.003019           1.089924   
38          105           150.003019           1.028133   
39          106           150.003019           1.330403   
40          107           150.003019           1.029456   
41          108           150.003019           1.573858   
42          109           150.003019           1.008214   
43          110           150.003019           1.031105   
44          112           150.003019           1.031105   

    HousekeepingBMP_BaseRisk  SWPlan_BaseRisk  BMPInspectionDeficiency_Rate  \
0                   0.963550         0.337498                         1.000   
1                   0.982110         0.382344                         1.000   
2                   0.394759         0.394759                         0.000   
3                   0.359765         0.359765                         0.000   
4                   1.091404         0.428664                         0.000   
5                   0.351029         0.351029                         0.000   
6                   0.452946         0.452946                         0.855   
7                   0.339225         0.339225                         0.000   
8                   0.323702         0.323702                         0.000   
9                   0.351230         0.351230                         0.000   
10                  0.380138         0.380138                         0.000   
11                  0.388487         0.388487                         0.000   
12                  0.401021         0.401021                         0.000   
13                  0.368134         0.368134                         0.000   
14                  0.364245         0.364245                         0.000   
15                  1.319

# BMP FEASIBILITY EVALUATION
Talk about it...

Global variables related to existing sampling data include:  
 - 
 - 

Defined several functions that will be used by BMP Option Evaluation. These include:  
 - 
 - 

In [20]:
'''
#############################################################################################################
#                    EVALUATE BASE BMP FEASIBILITY at each facility  
#                Write results to the base_bmp_feasibility_test_results table.
############################################################################################################ 

'''

'\n#############################################################################################################\n#                    EVALUATE BASE BMP FEASIBILITY at each facility  \n#                Write results to the base_bmp_feasibility_test_results table.\n############################################################################################################ \n\n'

In [21]:
# %%capture cap --no-stderr

print('\n******Evaluating Base BMP feasibility at facilities.******')
ShowCalculations = True #flag indicating if steps should be outputted
Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT

#Only analyze bmps at facilities we have normalized PEP data for. make list of these facilities.
ls_id = [np.asscalar(id) for id in pd_exPEP_norm['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):    
    if ShowCalculations: print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
    myBMPs = session.query(Base_BMPs)
    for aBMP in myBMPs:
        if ShowCalculations:print ('\n######Evaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id, '######')
        BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac.id, aBMP, ShowCalculations)
session.commit
winsound.Beep(250,1000)
print ('*****************************************************************')
print ('* Completed evaluating Base BMP feasibility                     *')
if Expr.CountEvalErrors() >0:
    print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
    print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
else:
    print ('No errors detected.')
print ('*****************************************************************')

# with open('C:\\Users\\JonHonda\\Desktop\\Crap\\output.txt', 'w') as f:
#     f.write(cap.stdout)
# f.close()


******Evaluating Base BMP feasibility at facilities.******

***Evaluating base bmp feasibiilty tests for facility:  Kalihi-Palama Bus & Paratransit Facility

######Evaluating feasibility of base_bmp:  Hydrodynamic Separation  ID:  1 ######

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=867962.0
    attempting to retrieve value fo

  Wrote to base_bmp_feasibility_test_results as recordID: 10

######Evaluating feasibility of base_bmp:  Bioinfiltration / Bioretention  ID:  5 ######

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 11

  Attempting eval of feasibility_test ID:  2
proccessing expression: Feas-2=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 12

  Attempting eval of feasibility_test ID:

       QUERY RESULT: Indus_WQFR=12.96
  eval(586.06*12.96 - 41.868)=7553.469599999999
  eval(867962.0>7553.469599999999)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 25

######Evaluating feasibility of base_bmp:  Roofing  ID:  9 ######

  Attempting eval of feasibility_test ID:  16
proccessing expression: Feas-16=EM_Area>0
    attempting to retrieve value for:  ('EM_Area', ['EM_Area', 'val', 'facility_chars', 'EM_Area', 'id', 'FLOAT'])
       QUERY RESULT: EM_Area=120.0
  eval(120.0>0)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 26

######Evaluating feasibility of base_bmp:  Repair Existing Pavement  ID:  10 ######

  Attempting eval of feasibility_test ID:  17
proccessing expression: Feas-17=Dmg_Pave>0
    attempting to retrieve value for:  ('Dmg_Pave', ['Dmg_Pave', 'val', 'facility_chars', 'Dmg_Pave', 'id', 'FLOAT'])
       QUERY RESULT: Dmg_Pave=0
  

    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a dynamic expression. Query for static expression using provided unique identifiers
       dynamic expression: BMP_Size(base_bmps~bmp_size_expression_id~bmp_name) =  static expression: bmp_size_expr_Sand Filtration
       Reentering EvalExpr...
proccessing expression: bmp_size_expr_Sand Filtration=Indus_WQV / 6
    attempting to retrieve value for:  ('Indus_WQV', ['Indus_WQV', 'val', 'facility_chars', 'Indus_WQV', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQV=77687.10417
  eval(77687.10417 / 6)=12947.850695000001
  eval(854990.0>12947.850695000001)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 58

######Evaluating feasibility of base_bmp:  Coagulation Enhanced Treatment  ID:  8 ######

  Atte

     This is a dynamic expression. Query for static expression using provided unique identifiers
       dynamic expression: BMP_Size(base_bmps~bmp_size_expression_id~bmp_name) =  static expression: bmp_size_expr_Hydrodynamic Separation
       Reentering EvalExpr...
proccessing expression: bmp_size_expr_Hydrodynamic Separation=50
  eval(50)=50
  eval(216424.0>50)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 76

######Evaluating feasibility of base_bmp:  Enhanced Media Filtration (Replaceable Cartridge)  ID:  2 ######

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='No'
  eval('No'=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as r

...

  Attempting eval of feasibility_test ID:  11
proccessing expression: Feas-11=Fac_Slope<0.08
    attempting to retrieve value for:  ('Fac_Slope', ['Fac_Slope', 'val', 'facility_chars', 'Fac_Slope', 'id', 'FLOAT'])
       QUERY RESULT: Fac_Slope=None
  FAULT!!!! Error occured while evaluating expression:  None<0.08 :  '<' not supported between instances of 'NoneType' and 'float'
  I will set evaluation to NoneType
  eval(None<0.08)=None
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1643

  Attempting eval of feasibility_test ID:  14
proccessing expression: Feas-14=Unpave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Unpave_Area', ['Unpave_Area', 'val', 'facility_chars', 'Unpave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Unpave_Area=None
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~

proccessing expression: Feas-5=Soil_Type!='Rock'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type=None
  eval(None!='Rock')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1661

  Attempting eval of feasibility_test ID:  6
proccessing expression: Feas-6=Soil_Type!='Clay'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type=None
  eval(None!='Clay')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1662

  Attempting eval of feasibility_test ID:  14
proccessing expression: Feas-14=Unpave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Unpave_Area', ['Unpave_Area', 'val', 'facility_chars', 'Unpave_Area', 'id', 'FLOAT

  FAULT!!!! Error occured while evaluating expression:  250.75*None + 13.75 :  unsupported operand type(s) for *: 'float' and 'NoneType'
  I will set evaluation to NoneType
  eval(250.75*None + 13.75)=None
  FAULT!!!! Error occured while evaluating expression:  None>None :  '>' not supported between instances of 'NoneType' and 'NoneType'
  I will set evaluation to NoneType
  eval(None>None)=None
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1671

######Evaluating feasibility of base_bmp:  Media Filtration (Pressure)  ID:  4 ######

  Attempting eval of feasibility_test ID:  9
proccessing expression: Feas-9=TFMR_Exist=='Yes'
    attempting to retrieve value for:  ('TFMR_Exist', ['TFMR_Exist', 'val', 'facility_chars', 'TFMR_Exist', 'id', 'FLOAT'])
       QUERY RESULT: TFMR_Exist=None
  eval(None=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1672

  Att

  I will set evaluation to NoneType
  eval(None>None)=None
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1686

######Evaluating feasibility of base_bmp:  Coagulation Enhanced Treatment  ID:  8 ######

  Attempting eval of feasibility_test ID:  9
proccessing expression: Feas-9=TFMR_Exist=='Yes'
    attempting to retrieve value for:  ('TFMR_Exist', ['TFMR_Exist', 'val', 'facility_chars', 'TFMR_Exist', 'id', 'FLOAT'])
       QUERY RESULT: TFMR_Exist=None
  eval(None=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1687

  Attempting eval of feasibility_test ID:  10
proccessing expression: Feas-10=DS_SS_Exist=='Yes'
    attempting to retrieve value for:  ('DS_SS_Exist', ['DS_SS_Exist', 'val', 'facility_chars', 'DS_SS_Exist', 'id', 'FLOAT'])
       QUERY RESULT: DS_SS_Exist=None
  eval(None=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wr

*****************************************************************
* Completed evaluating Base BMP feasibility                     *
81  errors were encountered. Review output to identify location(s)
Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:
*****************************************************************


In [22]:
'''
#############################################################################################################
#                           Make all combinations of base bmps  
#                     Write results to the combos bmp database table
############################################################################################################ 
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT
'''
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
winsound.Beep(250,1000)

get a coffee...this one takes a while!
 Making BMP Combos of length: 1
 Find max pollutant removal rates for each BMP Combo of length:  1
  Made  13  combos
 Making BMP Combos of length: 2
 Find max pollutant removal rates for each BMP Combo of length:  2
  Made  78  combos
 Making BMP Combos of length: 3
 Find max pollutant removal rates for each BMP Combo of length:  3
  Made  286  combos
 Making BMP Combos of length: 4
 Find max pollutant removal rates for each BMP Combo of length:  4
  Made  715  combos
 Making BMP Combos of length: 5
 Find max pollutant removal rates for each BMP Combo of length:  5
  Made  1287  combos
 Making BMP Combos of length: 6
 Find max pollutant removal rates for each BMP Combo of length:  6
  Made  1716  combos
 Making BMP Combos of length: 7
 Find max pollutant removal rates for each BMP Combo of length:  7
  Made  1716  combos
 Making BMP Combos of length: 8
 Find max pollutant removal rates for each BMP Combo of length:  8
  Made  1287  combos
 Making

In [23]:
'''
#############################################################################################################
#                           TUNE BMP FEASIBILITY
#           
############################################################################################################ 
CODE TO TUNE BMP FEASIBILITY/.
Use base bmp feasibility results for each facility.
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #insert/update combo data to Combo_BMP_Feasibility_Test_Results table

    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind).applymap(lambda el: 0.00 if el is None else el) #el = 0. if nonetype to represent no removal rate change

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #prepare dataframe to return:
    pd_temp = pd.concat([pd_rr, pd_sums], axis = 1)#make one dataframe containing combo bmp's removal rates and costs:

#     this one's for testing only (won't work with non-base bmps):
    pd_temp.insert(0,'Facility_Name',myFacility.Fac_Name) #include facility id column
    pd_temp.insert(3,'BMP_Name', session.query(Base_BMPs.bmp_name).filter(Base_BMPs.id == q.first().combos_bmp_id).first())    
    return (pd_temp)
    

def Eval_FacBMPCombos(aFac, ShowCalculations):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac.id, ShowCalculations)).set_index('base_bmp_id')
    if ShowCalculations: display (df)   
    df = df.loc[df['is_feasible'] == 1]
    if ShowCalculations:
        print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
        display (df)
    feas_ls = df.index#send feasible base bmp ids to list
    pd_temp = pd.DataFrame()

    for feas in feas_ls:
         pd_temp = pd.concat([pd_temp,Eval_FacBMPCombo(df,aFac, [feas])])            

    print ('      There are ', len(pd_temp), ' combinations of feasible Base BMPs.')
    return pd_temp
    
def Eval_All_FacBMPCombos(ShowCalculations):
    print ('Evaluating feasibile BMP Options for each facility:')
    #Only analyze bmps at facilities we have data for. make list of these facilities.
    ls_id = [np.asscalar(id) for id in pd_exMaxConcs['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    pd_BaseBMPCombos = pd.DataFrame() #create empty dataframe that we'll fill w/ the feasibile combos
    for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):        
#         Eval_FacBMPOptions(aFac, ShowCalculations)
        pd_BaseBMPCombos = pd.concat([pd_BaseBMPCombos, Eval_FacBMPCombos(aFac, ShowCalculations)])
    pd_BaseBMPCombos.reset_index(drop=True, inplace=True)
    return pd_BaseBMPCombos


#############################################################################################################
#                           EVALUATE FEASIBILITY OF BMP COMBOS 
#                   (DEFINE GLOBAL VARIABLE: pd_BaseBMPCombos)
############################################################################################################ 
ShowCalculations = True
start_time = time.time()
pd_BaseBMPCombos = Eval_All_FacBMPCombos(ShowCalculations)
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
display(pd_BaseBMPCombos)
# WRITE COMBOS RESULTS TO EXCEL FILE
xlsFile = 'C:\\Users\\JonHonda\\Desktop\\Crap\\Combos.xls'
print ('writing to excel file: ', xlsFile)
writer = pd.ExcelWriter(xlsFile)
pd_BaseBMPCombos.to_excel(writer,'Output')
writer.save()
session.commit()


Evaluating feasibile BMP Options for each facility:

***Evaluating feasible bmp combos for facility:  Kalihi-Palama Bus & Paratransit Facility ***
****Evaluating feasibile base bmps****

Evaluating base bmps for Facility: Kalihi-Palama Bus & Paratransit Facility
  Estimate CIP costs:
proccessing expression: cip_cost_expr_Hydrodynamic Separation=37800 + 27*Indus_WQFR + Indus_WQV
    attempting to retrieve value for:  ('Indus_WQFR', ['Indus_WQFR', 'val', 'facility_chars', 'Indus_WQFR', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQFR=12.96
    attempting to retrieve value for:  ('Indus_WQV', ['Indus_WQV', 'val', 'facility_chars', 'Indus_WQV', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQV=31774.3
  eval(37800 + 27*12.96 + 31774.3)=69924.22
  Estimate O&M costs:
proccessing expression: om_cost_expr_Hydrodynamic Separation=1200*Indus_WQFR
    attempting to retrieve value for:  ('Indus_WQFR', ['Indus_WQFR', 'val', 'facility_chars', 'Indus_WQFR', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQF

Facility_ID                                      base_bmp_name  \
base_bmp_id                                                                   
1                      1                            Hydrodynamic Separation   
2                      1  Enhanced Media Filtration (Replaceable Cartridge)   
3                      1                              Biofiltration (Vault)   
4                      1                        Media Filtration (Pressure)   
5                      1                     Bioinfiltration / Bioretention   
6                      1                                  Inlet Insert Unit   
7                      1                                    Sand Filtration   
8                      1                     Coagulation Enhanced Treatment   
9                      1                                            Roofing   
10                     1                           Repair Existing Pavement   
11                     1                           Oil and Water Separators   
12                     1                                  Vegetative Swales   
13                     1                                          Detention   

             calc_cip_cost  calc_om_cost  is_feasible  
base_bmp_id                                            
1                 69924.22  15552.000000            1  
2               2471438.84  18144.000000            1  
3               4100962.72  15552.000000            1  
4               4720601.56  18144.000000            1  
5                      NaN           NaN            0  
6                  7500.00   8100.000000            1  
7               2729241.20  95322.900000            1  
8                904929.96  18144.000000            1  
9                 26400.00      0.000000            1  
10                     NaN           NaN            0  
11                     NaN           NaN            0  
12                     NaN           NaN            0  
13              1817160.80  10591.433333            1

****These are the feasible base bmps. I'll use them to make combos:****


Facility_ID                                      base_bmp_name  \
base_bmp_id                                                                   
1                      1                            Hydrodynamic Separation   
2                      1  Enhanced Media Filtration (Replaceable Cartridge)   
3                      1                              Biofiltration (Vault)   
4                      1                        Media Filtration (Pressure)   
6                      1                                  Inlet Insert Unit   
7                      1                                    Sand Filtration   
8                      1                     Coagulation Enhanced Treatment   
9                      1                                            Roofing   
13                     1                                          Detention   

             calc_cip_cost  calc_om_cost  is_feasible  
base_bmp_id                                            
1                 69924.22  15552.000000            1  
2               2471438.84  18144.000000            1  
3               4100962.72  15552.000000            1  
4               4720601.56  18144.000000            1  
6                  7500.00   8100.000000            1  
7               2729241.20  95322.900000            1  
8                904929.96  18144.000000            1  
9                 26400.00      0.000000            1  
13              1817160.80  10591.433333            1

      There are  9  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Pearl City Bus Facility ***
****Evaluating feasibile base bmps****

Evaluating base bmps for Facility: Pearl City Bus Facility
  Estimate CIP costs:
proccessing expression: cip_cost_expr_Hydrodynamic Separation=37800 + 27*Indus_WQFR + Indus_WQV
    attempting to retrieve value for:  ('Indus_WQFR', ['Indus_WQFR', 'val', 'facility_chars', 'Indus_WQFR', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQFR=12.98321901
    attempting to retrieve value for:  ('Indus_WQV', ['Indus_WQV', 'val', 'facility_chars', 'Indus_WQV', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQV=77687.10417
  eval(37800 + 27*12.98321901 + 77687.10417)=115837.65108327
  Estimate O&M costs:
proccessing expression: om_cost_expr_Hydrodynamic Separation=1200*Indus_WQFR
    attempting to retrieve value for:  ('Indus_WQFR', ['Indus_WQFR', 'val', 'facility_chars', 'Indus_WQFR', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQFR=1

...

****These are the feasible base bmps. I'll use them to make combos:****


Empty DataFrame
Columns: [Facility_ID, base_bmp_name, calc_cip_cost, calc_om_cost, is_feasible]
Index: []

      There are  0  combinations of feasible Base BMPs.
--- 14.042999744415283 execution time in seconds ---


Facility_Name bmp_fingerprint  combos_bmp_id  \
0    Kalihi-Palama Bus & Paratransit Facility             |1|              1   
1    Kalihi-Palama Bus & Paratransit Facility             |2|              2   
2    Kalihi-Palama Bus & Paratransit Facility             |3|              3   
3    Kalihi-Palama Bus & Paratransit Facility             |4|              4   
4    Kalihi-Palama Bus & Paratransit Facility             |6|              6   
5    Kalihi-Palama Bus & Paratransit Facility             |7|              7   
6    Kalihi-Palama Bus & Paratransit Facility             |8|              8   
7    Kalihi-Palama Bus & Paratransit Facility             |9|              9   
8    Kalihi-Palama Bus & Paratransit Facility            |13|             13   
9                     Pearl City Bus Facility             |1|              1   
10                    Pearl City Bus Facility             |2|              2   
11                    Pearl City Bus Facility             |3|              3   
12                    Pearl City Bus Facility             |4|              4   
13                    Pearl City Bus Facility             |6|              6   
14                    Pearl City Bus Facility             |7|              7   
15                    Pearl City Bus Facility             |8|              8   
16                    Pearl City Bus Facility             |9|              9   
17                    Pearl City Bus Facility            |13|             13   
18              Kapaa Refuse Transfer Station             |6|              6   
19              Kapaa Refuse Transfer Station             |9|              9   
20              Kapaa Refuse Transfer Station            |10|             10   
21              Kapaa Refuse Transfer Station            |13|             13   
22           Kawailoa Refuse Transfer Station             |9|              9   
23           Kawailoa Refuse Transfer Station            |10|             10   
24           Kawailoa Refuse Transfer Station            |13|             13   
25              Keehi Refuse Transfer Station             |1|              1   
26              Keehi Refuse Transfer Station             |2|              2   
27              Keehi Refuse Transfer Station             |3|              3   
28              Keehi Refuse Transfer Station             |4|              4   
29              Keehi Refuse Transfer Station             |6|              6   
..                                        ...             ...            ...   
233         Kailua Wastewater Treatment Plant             |8|              8   
234         Kailua Wastewater Treatment Plant             |9|              9   
235         Kailua Wastewater Treatment Plant            |10|             10   
236         Kailua Wastewater Treatment Plant            |13|             13   
237    Sand Island Wastewater Treatment Plant             |1|              1   
238    Sand Island Wastewater Treatment Plant             |2|              2   
239    Sand Island Wastewater Treatment Plant             |3|              3   
240    Sand Island Wastewater Treatment Plant             |4|              4   
241    Sand Island Wastewater Treatment Plant             |6|              6   
242    Sand Island Wastewater Treatment Plant             |7|              7   
243    Sand Island Wastewater Treatment Plant             |8|              8   
244    Sand Island Wastewater Treatment Plant             |9|              9   
245    Sand Island Wastewater Treatment Plant            |10|             10   
246    Sand Island Wastewater Treatment Plant            |13|             13   
247        Wahiawa Wastewater Treatment Plant             |4|              4   
248        Wahiawa Wastewater Treatment Plant             |6|              6   
249        Wahiawa Wastewater Treatment Plant             |8|              8   
250        Wahiawa Wastewater Treatment Plant             |9|              9   
251        Wahiawa Wastewater Tr

writing to excel file:  C:\Users\JonHonda\Desktop\Crap\Combos.xls


In [24]:
# session.close()
# engine.dispose()